In [14]:
import pandas as pd
import duckdb

In [23]:
def pandas_csv():
    df = (
        pd.read_csv("data/measurements.txt", sep=";", header=None, names=["station_name", "measurement"], engine="pyarrow")
            .groupby("station_name")
            .agg(["min", "mean", "max"])
    )
    df.columns = df.columns.get_level_values(level=1)
    df = df.reset_index()
    df.columns = ["station_name", "min_measurement", "mean_measurement", "max_measurement"]
    df = df.sort_values("station_name")

    if PRINT != True:
        return
    print("{", end="")
    for row in df.itertuples(index=False):
        print(
            f"{row.station_name}={row.min_measurement:.1f}/{row.mean_measurement:.1f}/{row.max_measurement:.1f}",
            end=", "
        )
    print("\b\b} ")

In [24]:
def pandas_parquet():
    df = (
        pd.read_parquet("data/measurements.parquet", engine="pyarrow")
            .groupby("station_name")
            .agg(["min", "mean", "max"])
    )
    df.columns = df.columns.get_level_values(level=1)
    df = df.reset_index()
    df.columns = ["station_name", "min_measurement", "mean_measurement", "max_measurement"]
    df = df.sort_values("station_name")

    if PRINT != True:
        return
    print("{", end="")
    for row in df.itertuples(index=False):
        print(
            f"{row.station_name}={row.min_measurement:.1f}/{row.mean_measurement:.1f}/{row.max_measurement:.1f}",
            end=", "
        )
    print("\b\b} ")

In [25]:
def duckdb_csv():
    with duckdb.connect() as conn:
        data = conn.sql("""
            select
                station_name,
                min(measurement) as min_measurement,
                cast(avg(measurement) as decimal(8, 1)) as mean_measurement,
                max(measurement) as max_measurement
            from read_csv(
                'data/measurements.txt',
                header=false,
                columns={'station_name': 'varchar', 'measurement': 'decimal(8, 1)'},
                delim=';',
                parallel=true
            )
            group by station_name
            order by station_name
        """)

        if PRINT != True:
            return
        print("{", end="")
        for row in sorted(data.fetchall()):
            print(
                f"{row[0]}={row[1]}/{row[2]}/{row[3]}",
                end=", ",
            )
        print("\b\b} ")

In [26]:
def process_file(file_name = "data/measurements.txt"):
    result = dict()

    with open(file_name, "rb") as f:
        for line in f:
            location, measurement = line.split(b";")
            measurement = float(measurement)
            if location not in result:
                result[location] = [
                    measurement,
                    measurement,
                    measurement,
                    1,
                ]
            else:
                _result = result[location]
                if measurement < _result[0]:
                    _result[0] = measurement
                if measurement > _result[1]:
                    _result[1] = measurement
                _result[2] += measurement
                _result[3] += 1

    if PRINT != True:
        return
        
    print("{", end="")
    for location, measurements in sorted(result.items()):
        print(
            f"{location.decode('utf8')}={measurements[0]:.1f}/{(measurements[2] / measurements[3]) if measurements[3] !=0 else 0:.1f}/{measurements[1]:.1f}",
            end=", ",
        )
    print("\b\b} ")


In [33]:
import time

PRINT = False
def time_it(func):
    start = time.time()
    func()
    print(str(func) + " -> ", (time.time() - start) * 1000, "ms")

time_it(pandas_csv)
time_it(pandas_parquet)
time_it(duckdb_csv)
time_it(process_file)

<function pandas_csv at 0x7be34dec1e40> ->  105.87263107299805 ms
<function pandas_parquet at 0x7be34dec1ee0> ->  106.11891746520996 ms
<function duckdb_csv at 0x7be34dec25c0> ->  21.648168563842773 ms
<function process_file at 0x7be34dec22a0> ->  362.9734516143799 ms
